In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import selenium

In [2]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204%3A19&blank_scope=Latest"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')

In [13]:
#print(soup.prettify())

In [3]:
container_box = soup.find_all('div', class_='image_and_description_container')

In [4]:
#link = soup.find_all('li', class_='slide')
title = container_box[0].find_all('img')[1]['alt']
para = container_box[0].find_all('a')[0].text.strip()

In [31]:
# Create an executable path, initialize chrome driver
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path)
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
browser.click_link_by_id('full_image')
moreButton = browser.links.find_by_partial_text('more info')
moreButton.click()
featured_url = browser.html
featured_image = BeautifulSoup(featured_url, "html.parser")
jpl_image_url = featured_image.select_one("figure.lede a img").get("src")
full_feature_url = f"https://www.jpl.nasa.gov/{jpl_image_url}"
print(full_feature_url)
browser.quit()

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA17172_hires.jpg


https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA18295_hires.jpg
